Prompt 是大模型的核心，传统方式我们一般使用字符串拼接或者模版字符串来构造 prompt，而有了 langchain 后，我们可以构建可复用的 prompt 来让我们更工程化的管理和构建 prompt，从而制作更复杂的 chat bot

## 基础 prompt

首先我们学习基础的 `PromptTemplate` 来理解 langchain 中是如何构建和管理 prompt template。

`PromptTemplate` 是帮助我们定义一个包含变量的字符串模版，我们可以通过向该类的对象输入不同的变量值来生成模版渲染的结果。 这可以方便的让我们定义一组 prompt 模板，然后在运行时根据用户的输入动态地填充变量从而生成 prompt。

### 无变量 template

我们先从最基础的无变量 template 来逐步上手和理解


In [1]:
import { PromptTemplate } from "@langchain/core/prompts"

const greetingPrompt = new PromptTemplate({
  inputVariables: [],
  template: "hello world",
})
const formattedGreetingPrompt = await greetingPrompt.format()

console.log(formattedGreetingPrompt)


hello world


这里创建了一个 PromptTemplate 对象，并使用 format() 调用该对象的内容。

`PromptTemplate` 就是最基础的 template，我们不传入任何变量（`inputVariables: []`），这跟硬编码一个字符串没任何区别。 调用 prompt template 的方式就是 `format`，因为我们没有任何变量，也就没有任何参数。至于为什么叫做 format 相信会在接下来理解的更深刻。此处的 format 只是简单的返回 template 中的内容。

没有变量的 prompt template 使用的很少，这里主要以此帮助大家理解 template 的概念。


# 含变量的 template


In [2]:
const multiVariableGreetingPrompt = new PromptTemplate({
  inputVariables: ["timeOfDay", "name"],
  template: "good {timeOfDay}, {name} {{test}}",
})
const formattedMultiVariableGreeting = await multiVariableGreetingPrompt.format(
  {
    timeOfDay: "morning",
    name: "Kai",
  },
)

console.log(formattedMultiVariableGreeting)
// good morning, Kai {test}


good morning, Kai {test}


上述代码创建了一个 PromptTemplate 对象，内含两个变量，同时使用了`{{}}`来转译｛｝。

此外，你可能注意到其实创建 PromptTemplate 对象的过程中，部份信息的输出存在冗余，比如变量部分的`{}`其实已经表明了变量的性质和个数，因此我们可以更简便地创建 PromptTemplate 对象，langchain 可以自行推导完整的内容：


In [3]:
const autoInferTemplate = PromptTemplate.fromTemplate(
  "good {timeOfDay}, {name}",
)
console.log(autoInferTemplate.inputVariables)
// ['timeOfDay', 'name']

const formattedAutoInferTemplate = await autoInferTemplate.format({
  timeOfDay: "morning",
  name: "Kai",
})
console.log(formattedAutoInferTemplate)
// good morning, Kai


[ "timeOfDay", "name" ]
good morning, Kai


## 使用部分参数创建 template

我们并不需要一次性把所有变量都输入进去，在工程中，我们可能先获得某个参数，之后才能获得另一个参数。这里类似于函数式编程的概念，我们给需要两个参数的 prompt template 传递一个参数后，就会生成需要一个参数的 prompt template。

这里需要注意 `partial()` 和 `format()` 的使用

注意，如果未完整传参会报错`ReferenceError: formattedPrompt is not defined`。


In [2]:
const initialPrompt = new PromptTemplate({
  template: "这是一个{type}，它是{item}。",
  inputVariables: ["type", "item"],
})

const partialedPrompt = await initialPrompt.partial({
  type: "工具",
})
// console.log(formattedPrompt) 报错
const formattedPrompt = await partialedPrompt.format({
  item: "锤子",
})

console.log(formattedPrompt)
// 这是一个工具，它是锤子。

const formattedPrompt2 = await partialedPrompt.format({
  item: "改锥",
})

console.log(formattedPrompt2)
// 这是一个工具，它是改锥。


ReferenceError: formattedPrompt is not defined

## 使用动态填充参数

当我们需要，一个 prompt template 被 `format` 时，实时地动态生成参数时，我们可以使用函数来对 template 部分参数进行指定。


In [3]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString()
}

const promptWithDate = new PromptTemplate({
  template: "今天是{date}，{activity}。",
  inputVariables: ["date", "activity"],
})

const partialedPromptWithDate = await promptWithDate.partial({
  date: getCurrentDateStr,
})

const formattedPromptWithDate = await partialedPromptWithDate.format({
  activity: "我们去爬山",
})

console.log(formattedPromptWithDate)
// 输出: 今天是2023/7/13，我们去爬山。


今天是3/5/2025，我们去爬山。


注意，函数 `getCurrentDateStr` 是在 `format` 被调用的时候实时运行的，也就是可以在被渲染成字符串时获取到最新的外部信息。

目前这里不支持传入参数，如果需要参数，可以用 js 的闭包进行参数的传递。 假设我们有一个根据时间段（morning, afternoon, evening）返回不同问候语，并且需要带上当前时间的需求


In [5]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString()
}

function generateGreeting(timeOfDay) {
  return () => {
    const date = getCurrentDateStr()
    switch (timeOfDay) {
      case "morning":
        return date + " 早上好"
      case "afternoon":
        return date + " 下午好"
      case "evening":
        return date + " 晚上好"
      default:
        return date + " 你好"
    }
  }
}

const prompt = new PromptTemplate({
  template: "{greeting}!",
  inputVariables: ["greeting"],
})

const currentTimeOfDay = "morning"

const partialPrompt = await prompt.partial({
  greeting: generateGreeting(currentTimeOfDay),
})

const formattedPrompt = await partialPrompt.format()

console.log(formattedPrompt)
// 输出: 3/21/2024 下午好!


ReferenceError: PromptTemplate is not defined

## chat prompt

基础的 prompt template 算是开胃菜，因为 chat API 是目前跟 llm 交互的主流形式，`ChatPromptTemplate` 是最常用的工具。

在跟各种聊天模型交互的时候，在构建聊天信息时，不仅仅包含了像上文中的文本内容，也需要与每条消息关联的**角色信息**。 例如这条信息是由 人类、AI、还是给 chatbot 指定的 system 信息，这种**结构化的消息输入**有助于模型更好地理解对话的上下文和流程，从而生成更准确、更自然的回应。

为了方便地构建和处理这种结构化的聊天消息，LangChain 提供了几种与聊天相关的提示模板类，如 `ChatPromptTemplate`、`SystemMessagePromptTemplate`、`AIMessagePromptTemplate` 和 `HumanMessagePromptTemplate`。

其中后面三个分别对应了一段 ChatMessage 不同的角色。在 OpenAI 的定义中，每一条消息都需要跟一个 role 关联，标识消息的发送者。**角色的概念对 LLM 理解和构建整个对话流程非常重要**，相同的内容由不同的 role 发送出来的意义是不同的。

- `system` 角色的消息通常用于设置对话的**上下文**或指定模型采取特定的**行为模式**。这些消息不会直接显示在对话中，但它们对模型的行为有**指导作用**。 可以理解成模型的元信息，**权重非常高**，在这里有效的构建 prompt 能取得非常好的效果。
- `user` 角色代表**真实用户**在对话中的发言。这些消息通常是问题、指令或者评论，反映了用户的意图和需求。
- `assistant` 角色的消息代表**AI 模型**的回复。这些消息是模型根据 system 的指示和 user 的输入生成的。

我们以一个基础的翻译 chatbot 来讲解这几个常见 chat template，我们先构建一个 system message 来给 llm 指定核心的准则


In [1]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts"

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。",
)


然后构建一个用户输入的信息


In [2]:
import { HumanMessagePromptTemplate } from "@langchain/core/prompts"

const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate(
  "请翻译这句话：{text}",
)


然后将这两个信息组合起来，形成一个对话信息


In [6]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

const chatPrompt = ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
])


ChatPromptTemplate {
  lc_serializable: true,
  lc_kwargs: {
    inputVariables: [ "source_lang", "target_lang", "text" ],
    promptMessages: [
      SystemMessagePromptTemplate {
        lc_serializable: true,
        lc_kwargs: { prompt: [PromptTemplate] },
        lc_runnable: true,
        name: undefined,
        lc_namespace: [ "langchain_core", "prompts", "chat" ],
        inputVariables: [ "source_lang", "target_lang" ],
        additionalOptions: {},
        prompt: PromptTemplate {
          lc_serializable: true,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: undefined,
          lc_namespace: [Array],
          inputVariables: [Array],
          outputParser: undefined,
          partialVariables: undefined,
          templateFormat: "f-string",
          template: "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。",
          validateTemplate: true
        },
        messageClass: undefined,
        chatMessageClass: undefined
      },
      

然后我们就可以用一个 `formatMessages` 来格式化整个对话信息


In [7]:
const formattedChatPrompt = await chatPrompt.formatMessages({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})
formattedChatPrompt


[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你是一个专业的翻译员，你的任务是将文本从中文翻译成法语。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你是一个专业的翻译员，你的任务是将文本从中文翻译成法语。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "请翻译这句话：你好，世界",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "请翻译这句话：你好，世界",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

In [ ]:
const systemTemplate =
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。"
const humanTemplate = "请翻译这句话：{text}"

const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
])


然后我们就可以快速组装起一个简单的 chain 来测试一下，注意这里只需要引入 ChatPromptTemplate 就行了。


In [2]:
import { ChatOpenAI } from "@langchain/openai"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { ChatPromptTemplate } from "@langchain/core/prompts"

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://yunwu.ai/v1",
  },
  modelName: "gpt-4o-mini",
})

const systemTemplate =
  "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。"
const humanTemplate = "请翻译这句话：{text}"

const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
])

const outputPraser = new StringOutputParser()

const chain = chatPrompt.pipe(chatModel).pipe(outputPraser)

await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})
// "Bonjour, le monde."


"Bonjour, le monde."

## 组合多个 Prompt

在实际工程中，我们可能会根据多个变量，根据多个外界环境去构造一个很复杂的 prompt，这里就是`PipelinePromptTemplate` 的应用场景。 我可以用将多个独立的 template 构建成一个完整且复杂的 prompt，这样可以提高独立 prompt 的复用性，进一步增强模块化带来的优势。

在 `PipelinePromptTemplate` 有两个核心的概念：

- `pipelinePrompts`，一组 object，每个 object 表示 `prompt` 运行后赋值给 `name` 变量
- `finalPrompt`，表示最终输出的 prompt

我们还是少废话，直接看代码


In [3]:
import { PipelinePromptTemplate, PromptTemplate } from "@langchain/core/prompts"

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString()
}

const fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
根据上下文，完成主人的需求
{task}`)

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}")
const periodPrompt = await datePrompt.partial({
  date: getCurrentDateStr,
})

const infoPrompt = PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}")

const taskPrompt = PromptTemplate.fromTemplate(`
我想吃 {period} 的 {food}。 
再重复一遍我的信息 {info}`) //这里的info和fullPrompt中的info指向了同一个变量

// 构建 PipelinePromptTemplate
//使用多个prompt，通过替换变量来构建一个复杂的prompt
const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    {
      name: "date",
      prompt: periodPrompt,
    },
    {
      name: "info",
      prompt: infoPrompt,
    },
    {
      name: "task",
      prompt: taskPrompt,
    },
  ],
  finalPrompt: fullPrompt, //指向fullPrompt
})

const formattedPrompt = await composedPrompt.format({
  period: "早上",
  name: "张三",
  gender: "male",
  food: "lemon",
})

console.log(formattedPrompt)



你是一个智能管家，今天是 3/6/2025，现在是 早上，你的主人的信息是姓名是 张三, 性别是 male, 
根据上下文，完成主人的需求

我想吃 早上 的 lemon。 
再重复一遍我的信息 姓名是 张三, 性别是 male


这里有几个需要注意的地方

- 一个变量可以多次复用，例如外界输入的 `period` 在 `periodPrompt` 和 `taskPrompt` 都被使用了
- `pipelinePrompts` 中的变量可以被引用，例如我们在 `taskPrompt` 使用了 `infoPrompt` 的运行结果
- 支持动态自定义和 partial。例子中我们也涉及到了这两种特殊的 template
- langchain 会自动分析 pipeline 之间的依赖关系，尽可能的进行并行化来提高运行速度

有了 `pipelinePrompts` 我们可以极大程度的复用和管理我们的 prompt template，从而让 llm app 的开发更加工程化。
